# Proyecto 2: Filtros de imágenes 
## Clasificación de Galaxias con Deep Learning 
 Carol Edith Quiñones Sánchez 

#### Para diseñar una red neuronal capaz de clasificar 3 tipos de galaxias, elípticas, espirales y barradas es necesario usar filtros de textura con arquitectura de CNN. Esto es necesario para poder distinguir la forma de cada galaxía y así hacer una ditinción entre cada tipo.

In [182]:
### ---- Importación de librerias --- ###
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Conv2D
from sklearn.metrics import r2_score

In [183]:
### --- Configuration de datos --- ###
DATA_DIR = "galaxys1"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 5 # número de clases 

In [184]:
# visualización de carpetas que contienen la imagenes 
os.listdir('galaxys1')

['.ipynb_checkpoints', 'Barradas', 'Elipticas', 'Espirales']

In [185]:
### --- Pre-procesamiento de filtros clásicos --- ###
def apply_classical_filters(img):
    # Escala de gris para imágenes 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Sobel X + Y (3x3) resalta bordes horizontales y verticales
    sobel_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    
    # Laplaciano (3x3) detecta direcciones de cambio de intensidad
    laplacian = cv2.Laplacian(gray, cv2.CV_32F, ksize=3)
    
    # Ecualización adaptativa de histograma (CLAHE) mejora contraste en regiones debiles de luminosidad
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    hc = clahe.apply(gray)
    # Blur Gaussiano (5x5) elimina ruido de fondo antes de detectar bordes
    blurred = cv2.GaussianBlur(img, (5,5), sigmaX=1.0)
    
     # Normalizar cada canal al rango [0,1]
    sobel_x = cv2.normalize(sobel_x, None, 0, 1, cv2.NORM_MINMAX)
    sobel_y = cv2.normalize(sobel_y, None, 0, 1, cv2.NORM_MINMAX)
    laplacian = cv2.normalize(laplacian, None, 0, 1, cv2.NORM_MINMAX)
    hc = hc.astype(np.float32) / 255.0
    blurred = blurred.astype(np.float32) / 255.0
    # Reescalar imagen original a [0,1]
    img_norm = img.astype(np.float32) / 255.0
    
    # Concatenar canales: RGB + SobelX + SobelY + Laplacian + CLAHE_gray
    combined = np.concatenate([
        img_norm,                            # 3 canales
        np.expand_dims(sobel_x, -1),        # 1 canal
        np.expand_dims(sobel_y, -1),        # 1 canal
        np.expand_dims(laplacian, -1),      # 1 canal
        np.expand_dims(hc, -1)              # 1 canal
    ], axis=-1)
    
    return combined

In [186]:
CHANNELS = 3 + 4  # 3 RGB + 4 filtros clásicos

ds = (
    paths_ds
    .shuffle(buffer_size=len(file_paths), reshuffle_each_iteration=True)
    .map(lambda p, l: tf.py_function(
             func=load_and_preprocess,
             inp=[p, l],
             Tout=(tf.float32, tf.int32)),
         num_parallel_calls=tf.data.AUTOTUNE)
    # <- Aquí corregimos la forma faltante:
    .map(lambda img, lbl: (
         tf.ensure_shape(img, IMG_SIZE + (CHANNELS,)),
         tf.ensure_shape(lbl, [])
    ))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [187]:
def load_and_preprocess(path, label):
    # 1) Extrae el string del tensor
    path_str = path.numpy().decode("utf-8")
    lbl      = label.numpy()           # si quieres un numpy int
    
    # 2) Lee y procesa la imagen
    img = cv2.imread(path_str)
    img = cv2.resize(img, IMG_SIZE)
    
    # (si usas filtros clásicos)
    filtered = apply_classical_filters(img)
    
    # 3) Devuelve NumPy arrays de tipo correcto
    return filtered.astype(np.float32), np.int32(lbl)

##### (3x3) detecta bordes y esquinas (Workhorse de la mayoria de arquitecturas).
##### (5x5) capta texturas de mayor escala.

In [188]:
paths_ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))

ds = (
    paths_ds
    .shuffle(buffer_size=len(file_paths), reshuffle_each_iteration=True)
    .map(lambda p, l: tf.py_function(
             func=load_and_preprocess,
             inp=[p, l],
             Tout=(tf.float32, tf.int32)),
         num_parallel_calls=tf.data.AUTOTUNE)
    .map(lambda img, lbl: (
         tf.ensure_shape(img, IMG_SIZE + (CHANNELS,)),
         tf.ensure_shape(lbl, [])))
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [189]:
### --- Cargar rutas de archivos y etiquetas --- ###
class_names = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
file_paths = []
labels = []
for idx, cls in enumerate(class_names):
    cls_dir = os.path.join(DATA_DIR, cls)
    for fname in os.listdir(cls_dir):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            file_paths.append(os.path.join(cls_dir, fname))
            labels.append(idx)


In [190]:
### --- Convertir a Dataset --- ###
paths_ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
ds = paths_ds.shuffle(len(file_paths)) \
             .map(lambda p, l: tf.py_function(func=load_and_preprocess, inp=[p, l], Tout=(tf.float32, tf.int32)),
                  num_parallel_calls=tf.data.AUTOTUNE) \
             .batch(BATCH_SIZE) \
             .prefetch(tf.data.AUTOTUNE)

In [191]:
### --- Modelo CNN simple --- ###
input_shape = IMG_SIZE + (3 + 4,)  # 3 canales RGB + 4 filtros clásicos
inputs = layers.Input(shape=input_shape)


In [192]:
x = layers.Conv2D(32, 3, padding="same", activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)

In [193]:
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
x = layers.MaxPooling2D()(x)

In [194]:
x = layers.Conv2D(128, 3, dilation_rate=2, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)

In [195]:
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

In [196]:
model = models.Model(inputs, outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.build((None, IMG_SIZE[0], IMG_SIZE[1], CHANNELS))
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 224, 224, 7)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 224, 224, 32)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 224, 224, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_32 (Conv2D)                   │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_33 (Conv2D)                   │ (None, 112, 112, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_34 (Conv2D)                   │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 175,781 (686.64 KB)

 Trainable params: 175,333 (684.89 KB)

 Non-trainable params: 448 (1.75 KB)

In [197]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
    run_eagerly=True
)

In [198]:
model.fit(ds, epochs=20)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.1945 - loss: 1.6938
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5685 - loss: 1.2320
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6650 - loss: 1.0394
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6001 - loss: 1.0012
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6616 - loss: 0.8278
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7092 - loss: 0.7597
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6195 - loss: 0.8419
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6885 - loss: 0.8003
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7820 - loss: 0.6364
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7864 - loss: 0.5993
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7091 - loss: 0.6866
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.7360 - loss: 0.6404
Epoch 13/20
5

In [199]:
# Guardar modelo
model.save("galaxy_classifier_with_filters.h5")

In [200]:
### --- Parámetros --- ###
IMG_SIZE  = (224, 224)
CHANNELS  = 3 + 4   # 3 RGB + 4 filtros clásicos
CLASS_NAMES = ["Barrada", "Espiral", "Eliptica", "..."]  # tus clases

In [201]:
### --- Carga el modelo --- ###
model = load_model("galaxy_classifier_with_filters.h5")

In [202]:
### --- Define la misma función de filtros --- ### 
def apply_classical_filters(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.normalize(cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3),
                             None, 0, 1, cv2.NORM_MINMAX)
    sobel_y = cv2.normalize(cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3),
                             None, 0, 1, cv2.NORM_MINMAX)
    laplacian = cv2.normalize(cv2.Laplacian(gray, cv2.CV_32F, ksize=3),
                               None, 0, 1, cv2.NORM_MINMAX)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gray)
    clahe = clahe.astype(np.float32) / 255.0

    img_norm = img.astype(np.float32) / 255.0
    blurred  = cv2.GaussianBlur(img_norm, (5,5), sigmaX=1.0)

    return np.concatenate([
        img_norm,
        sobel_x[...,None],
        sobel_y[...,None],
        laplacian[...,None],
        clahe[...,None]
    ], axis=-1)

In [208]:
### --- Predicción de una imagen --- ### 
def predict_image(path_to_image):
    # Redimenciona
    img = cv2.imread(path_to_image)
    img = cv2.resize(img, IMG_SIZE)
    # Aplicación de filtros y normalización
    x   = apply_classical_filters(img)
    # Crea batch de tamaño 1
    x   = np.expand_dims(x, axis=0)   # shape (1, 224,224,7)
    # Predección
    probs = model.predict(x)[0]       # vector de probabilidades
    idx   = np.argmax(probs)          # índice de la clase más probable
    return CLASS_NAMES[idx], probs[idx]

In [212]:
### --- EJEMPLO --- ###
imagen = "galaxys1/Elipticas/Colliding Galaxies NGC 1410 and NGC 1409.png"
clase, confianza = predict_image(imagen)
print(f"Predicción: {clase} (confianza={confianza*100:.1f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicción: Eliptica (confianza=82.8%)
